In [ ]:
# @title Youtube download Server
addr = "" # @param {type:"string"}
tunnel = "srv" # @param ["lhr","pinggy","ngrok", "srv", "cloudflare"]

token = "" # @param {type:"string"}

print('please wait ...')
import os, re, time
from subprocess import Popen, PIPE
os.system('pip install yt-dlp')
os.system(f'yt-dlp -f 18 -o a.mp4 {addr}')
os.system('apt install nginx')

#os.system('cp -f -y /content/a.mp4 /var/www/html/a.mp4')
txt=''

with open('/etc/nginx/sites-enabled/default','r') as f:
 txt=f.read()
txt=txt.replace('index index.html index.htm','index a.mp4 index.html index.htm')
txt=txt.replace('root /var/www/html;','root /content;')
with open('/etc/nginx/sites-enabled/default','w') as f:
 f.write(txt)
os.system('service nginx restart')

if tunnel=='ngrok':
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
  !chmod +x ./ngrok

if tunnel=='ngrok':
  with open('/root/ngrok.yml','w') as f:
    f.write(f'''authtoken: {token}
region: us
update: false
update_channel: stable
web_addr: localhost:8098
tunnels:
  v2ray:
    addr: 80
    proto: http
    inspect: false
      ''')

  Popen("./ngrok start --config /root/ngrok.yml --all ".split(), stdout=PIPE, stdin=PIPE, stderr=PIPE, shell=False)
  time.sleep(5)
  !curl localhost:8098/api/tunnels > data.json
  data=json.loads(open('data.json','r').read())
  url=data['tunnels'][0]['public_url'][8:]


#from google.colab.output import eval_js
#url=eval_js("google.colab.kernel.proxyPort(9910)")

if tunnel=='lhr':
  process = Popen("ssh -R 80:localhost:9910 nokey@localhost.run -o StrictHostKeyChecking=no".split(), stdout=PIPE)
  output = process.stdout.readline()
  output = process.stdout.readline()
  addr=re.findall("https://(.*?.lhr.life)",str(output))
  url=addr[0]

if tunnel=='pinggy':
  process = Popen("ssh -p 443 -R0:localhost:9910 a.pinggy.io -o StrictHostKeyChecking=no ".split(), stdout=PIPE)
  output = str(process.stdout.readline())
  output += str(process.stdout.readline())
  output += str(process.stdout.readline())
  output += str(process.stdout.readline())
  addr=re.findall("https://(.*?.a.free.pinggy.online)",output)
  url=addr[1]

if tunnel=='srv':
  !ssh-keygen -t ed25519 -N '' -f ~/.ssh/id_rsa <<< y
  process=Popen('ssh srv.us -R 1:localhost:80 -o StrictHostKeyChecking=no'.split(), stdout=PIPE)
  output = str(process.stdout.readline())
  addr=re.findall("(https://.*?.srv.us)",output)
  url=addr[0]

print (url)